In [4]:
import sys
sys.path.insert(0, '../')
#from models.model_mrcnn import _default_mrcnn_config, build_default
import torchvision
import torch
from models_pytorch_lightning.generalized_mask_rcnn_pl import LitMaskRCNN
import torchvision.ops.boxes as bops
import torchvision
from torchvision import transforms
import torch
from PIL import Image
import cv2
import numpy as np
import os
import pandas as pd

In [5]:
model_name = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/runpod_mrcnn_models/yp2mf3i8_epoch=108-step=872.ckpt"
model = LitMaskRCNN.load_from_checkpoint(model_name)

In [6]:
class_names = ['Cored', 'Diffuse', 'Coarse-Grained', 'CAA']

In [8]:
def crop_and_resize(image, crop_size=(768, 768), resize_size=(1024, 1024)):
    """
    Crop an image into 4 equal parts and resize each crop.
    
    Args:
        image: Input image as a NumPy array (e.g., 1536x1536x3).
        crop_size: Tuple (height, width) for each crop.
        resize_size: Tuple (height, width) for resizing each crop.
        
    Returns:
        List of resized crops.
    """
    h, w = crop_size
    crops = []
    
    # Define crop coordinates for 4 regions
    for i in range(0, image.shape[0], h):
        for j in range(0, image.shape[1], w):
            crop = image[i:i+h, j:j+w]
            resized_crop = cv2.resize(crop, resize_size, interpolation=cv2.INTER_LINEAR)
            crops.append(resized_crop)
    
    return crops



def prepare_input(image):
    image_float_np = np.float32(image) / 255
    # define the torchvision image transforms
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(1024, interpolation=Image.LANCZOS),
        torchvision.transforms.ToTensor(),
    ])
    input_tensor = transform(transforms.ToPILImage()(image))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_tensor = input_tensor.to(device)
    # Add a batch dimension:
    input_tensor = input_tensor.unsqueeze(0)
    return input_tensor, image_float_np



def get_outputs_nms(input_tensor, model, score_threshold = 0.5, iou_threshold = 0.5):
    #mask_list = []
    #label_list = []
    #score_list =[]
    #box_list = []
    with torch.no_grad():
        # forward pass of the image through the modle
        outputs = model(input_tensor)
    boxes = outputs[0]['boxes']
    labels = outputs[0]['labels']
    scores = outputs[0]['scores']
    masks = outputs[0]['masks']
    #print(boxes.shape,labels.shape, masks.shape)
    # Apply score threshold
    keep = scores > score_threshold
    boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
    #print(boxes.shape,labels.shape, masks.shape)
    #print(keep)
    keep = torchvision.ops.nms(boxes, scores, iou_threshold)
    boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
    #print(keep)
    #print(boxes.shape,labels.shape, masks.shape)
    scores = list(scores.detach().cpu().numpy())
    #print(scores)
    masks = list((masks>0.5).detach().cpu().numpy())
    #masks = list((masks>0.5).squeeze().detach().cpu().numpy())
    #print(masks.shape)
    #boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in boxes.detach().cpu()]
    boxes = [[int(i[0]), int(i[1]), int(i[2]), int(i[3])]  for i in boxes.detach().cpu()]
    labels = list(labels.detach().cpu().numpy())
    labels = [class_names[i-1] for i in labels]
    return masks, boxes, labels, scores



def yolo_to_box_coordinates(image_width, image_height, yolo_bboxes):
    """
    Convert YOLO normalized bounding box format to (x_min, y_min, x_max, y_max) format.

    Args:
        image_width (int): Width of the original image.
        image_height (int): Height of the original image.
        yolo_bbox (list or tuple): A list or tuple containing YOLO format [x_center, y_center, width, height].
                                   All values should be normalized (between 0 and 1).

    Returns:
        tuple: Bounding box in (x_min, y_min, x_max, y_max) format.
    """
    output_bbox = []
    for yolo_bbox in yolo_bboxes:
        x_center, y_center, width, height = yolo_bbox

        # Convert YOLO normalized coordinates to absolute coordinates
        x_center_abs = x_center * image_width
        y_center_abs = y_center * image_height
        width_abs = width * image_width
        height_abs = height * image_height

        # Calculate the corner points of the bounding box
        x_min = int(x_center_abs - width_abs / 2)
        y_min = int(y_center_abs - height_abs / 2)
        x_max = int(x_center_abs + width_abs / 2)
        y_max = int(y_center_abs + height_abs / 2)
        output_bbox.append([x_min, y_min, x_max, y_max])
    return output_bbox



def transform_yolo_boxes(yolo_boxes, crop_origin, crop_size, resize_size):
    """
    Adjust YOLO bounding boxes for a cropped and resized image.
    
    Args:
        yolo_boxes: List of YOLO boxes [label, x_center, y_center, width, height] normalized for the original image.
        crop_origin: (x_offset, y_offset) top-left origin of the crop in the original image.
        crop_size: (crop_width, crop_height) size of the crop.
        resize_size: (resize_width, resize_height) target size for resizing.
    
    Returns:
        List of transformed YOLO boxes for the crop.
    """
    x_offset, y_offset = crop_origin
    crop_width, crop_height = crop_size
    resize_width, resize_height = resize_size

    transformed_boxes = []
    for box in yolo_boxes:
        label, x_center, y_center, width, height = box
        
        # Denormalize coordinates to original image size
        x_center_orig = x_center * 1536
        y_center_orig = y_center * 1536
        width_orig = width * 1536
        height_orig = height * 1536

        # Check if the box center lies within the crop
        if (x_offset <= x_center_orig <= x_offset + crop_width) and (y_offset <= y_center_orig <= y_offset + crop_height):
            # Adjust coordinates relative to the crop
            x_center_crop = x_center_orig - x_offset
            y_center_crop = y_center_orig - y_offset

            # Rescale to the resized crop
            x_center_resized = (x_center_crop / crop_width) * resize_width
            y_center_resized = (y_center_crop / crop_height) * resize_height
            width_resized = (width_orig / crop_width) * resize_width
            height_resized = (height_orig / crop_height) * resize_height

            # Normalize back to [0, 1] for the resized image
            x_center_norm = x_center_resized / resize_width
            y_center_norm = y_center_resized / resize_height
            width_norm = width_resized / resize_width
            height_norm = height_resized / resize_height

            transformed_boxes.append([label, x_center_resized, y_center_resized, width_resized, height_resized])

    return transformed_boxes


def bbox_iou(box1, box2):
    """
    Returns the IoU of two bounding boxes
    """
    # Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1
    b2_x1, b2_y1, b2_x2, b2_y2 = box2

    # get the corrdinates of the intersection rectangle
    inter_rect_x1 = np.max(b1_x1, b2_x1)
    inter_rect_y1 = np.max(b1_y1, b2_y1)
    inter_rect_x2 = np.min(b1_x2, b2_x2)
    inter_rect_y2 = np.min(b1_y2, b2_y2)
    # Intersection area
    inter_area = np.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * np.clamp(
        inter_rect_y2 - inter_rect_y1 + 1, min=0
    )
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)
    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)
    return iou

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval().to(device)

LitMaskRCNN(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256,

In [10]:
path = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/labels"
label_list = os.listdir(path)

In [11]:
label_list

['cw18-015_0_10_15.txt',
 'cw17-018_0_8_27.txt',
 'NA4963UCDtemporalgyri _4G8_0_6_32.txt',
 '48283-3_UTSW_temporal_6E10_0_6_32.txt',
 'cw17-023_0_25_22.txt',
 'cw18-015_0_20_27.txt',
 'NA4092UCDtemporalgyri _4G8_0_25_19.txt',
 '48283-3_UTSW_temporal_6E10_0_24_15.txt',
 '48283-3_UTSW_temporal_6E10_0_7_35.txt',
 'cw15-019_0_0_10.txt',
 '48283-3_UTSW_temporal_6E10_0_5_12.txt',
 'NA5012UCDtemporalgyri _4G8_0_8_31.txt',
 'NA5012UCDtemporalgyri _4G8_0_21_31.txt',
 'cw18-015_0_23_27.txt',
 'NA4003UCDtemporalgyri _4G8_0_19_10.txt',
 'cw17-023_0_24_33.txt',
 'cw18-002_0_23_23.txt',
 'cw17-022_0_12_32.txt',
 'cw18-002_0_10_32.txt',
 '51266-3_UTSW_temporal_6E10_0_9_31.txt',
 'NA4996UCDtemporalgyri _4G8_0_10_23.txt',
 '48283-3_UTSW_temporal_6E10_0_25_30.txt',
 'cw17-023_0_10_20.txt',
 '51399-3_UTSW_temporal_6E10_0_13_14.txt',
 '48308-3_UTSW_temporal_6E10_0_1_14.txt',
 'NA4963UCDtemporalgyri _4G8_0_11_20.txt',
 'cw17-023_0_22_20.txt',
 'NA5012UCDtemporalgyri _4G8_0_25_33.txt',
 '51266-3_UTSW_tempor

In [12]:
img_folder =  "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/images"
img_names =  [ os.path.join(img_folder,label_list[i].replace(".txt",".jpg")) for i in range(len(label_list))]

In [13]:
len(img_names)

659

In [14]:
img_names_list2 = []
len_gt_labels=[]
for idx, img_name in enumerate(img_names):
    label_path = os.path.join(path, label_list[idx])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    gt_labels = gt_boxes[:,0]
    img_names_list2.append(img_name)
    len_gt_labels.append(len(gt_labels))

In [113]:
gt_count_df = pd.DataFrame({"img_name":img_names_list2, "gt":len_gt_labels})

In [114]:
img_names_list2 = []
len_pred_labels=[]
for idx, img_name in enumerate(img_names):
    label_path = os.path.join(path, label_list[idx])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    gt_labels = gt_boxes[:,0]
    img_names_list2.append(img_name)
    len_gt_labels.append(len(gt_labels))

In [116]:
img_names_list = []
pred_label_list = []
act_label_list = []
iou_list = []
img_names_list2 = []
len_pred_labels=[]
len_gt_labels=[]
core_gt = []
caa_gt =[]
core_pred = []
caa_pred =[]


for idx, img_name in enumerate(img_names):
    image = np.array(Image.open(img_name))
    input_tensor, image_float_np = prepare_input(image)
    masks, boxes, labels, scores = get_outputs_nms(input_tensor, model, 0.1, 0.5)
    label_path = os.path.join(path, label_list[idx])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    gt_labels = gt_boxes[:,0]
    gt_boxes_coords = yolo_to_box_coordinates(1024, 1024, gt_boxes[:,1:])
    pred_matched = []
    for i in range(len(gt_boxes_coords)):
        for j in range(len(boxes)):
            #iou_val = bbox_iou(gt_boxes_coords[i], boxes[j])
            #print(iou_val)
            box1 = torch.tensor([gt_boxes_coords[i]], dtype=torch.float)
            box2 = torch.tensor([boxes[j]], dtype=torch.float)
            iou = bops.box_iou(box1, box2)
            if iou>0:
                img_names_list.append(img_name)
                pred_label_list.append(labels[j])
                act_label_list.append(gt_labels[i])
                iou_list.append(iou)
                pred_matched.append(j)
    #d = len([ x for x in labels if x=="Diffuse" ])
    img_names_list2.append(img_name)
    len_gt_labels.append(len(gt_labels))
    t = len([ x for x in gt_labels if x==0 ])
    caa_gt.append(t)
    core_gt.append(len(gt_labels)-t)
    t = len([ x  for x in labels if x=="CAA"])
    caa_pred.append(t)
    core_pred.append(len(boxes)-t)
    len_pred_labels.append(len(boxes))



In [54]:
# Crop based 
img_names_list = []
pred_label_list = []
act_label_list = []
iou_list = []
img_names_list2 = []
len_pred_labels=[]
len_gt_labels=[]
core_gt = []
caa_gt =[]
core_pred = []
caa_pred =[]
crop_sizes = [
    ((0, 0), (768, 768)),  # Top-left
    ((768, 0), (768, 768)),  # Top-right
    ((0, 768), (768, 768)),  # Bottom-left
    ((768, 768), (768, 768)),  # Bottom-right
]
resize_size = (1024, 1024)

for idx, img_name in enumerate(img_names):
    image = np.array(Image.open(img_name))
    crops = crop_and_resize(image, crop_size=(768, 768), resize_size=(1024, 1024))
    label_path = os.path.join(path, label_list[idx])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    gt_labels = gt_boxes[:,0]
    #gt_boxes_coords = yolo_to_box_coordinates(1536, 1536, gt_boxes[:,1:])
    for crop, crop_size in zip(crops,crop_sizes):
        input_tensor, image_float_np = prepare_input(crop)
        masks, boxes, labels, scores = get_outputs_nms(input_tensor, model, 0.4, 0.5)
        pred_matched = []
        transformed_boxes = transform_yolo_boxes(gt_boxes, crop_size[0], crop_size[1], resize_size)
        #print(len(transformed_boxes))
        #gt_labels = transformed_boxes[:,0]
        #gt_boxes_coords = transformed_boxes[:,1:]
        for i in range(len(transformed_boxes)):
            for j in range(len(boxes)):
                #iou_val = bbox_iou(gt_boxes_coords[i], boxes[j])
                #print(iou_val)
                box1 = torch.tensor([transformed_boxes[i][1:]], dtype=torch.float)
                box2 = torch.tensor([boxes[j]], dtype=torch.float)
                print(box1)
                print(box2)
                iou = bops.box_iou(box1, box2)
                if iou>0:
                    img_names_list.append(img_name)
                    pred_label_list.append(labels[j])
                    act_label_list.append(transformed_boxes[i][0])
                    iou_list.append(iou)
                    pred_matched.append(j)
        #d = len([ x for x in labels if x=="Diffuse" ])
    img_names_list2.append(img_name)
    len_gt_labels.append(len(gt_labels))
    t = len([ x for x in gt_labels if x==0 ])
    caa_gt.append(t)
    core_gt.append(len(gt_labels)-t)
    t = len([ x  for x in labels if x=="CAA"])
    caa_pred.append(t)
    core_pred.append(len(boxes)-t)
    len_pred_labels.append(len(boxes))

tensor([[604.0000, 449.3333,  98.6667,  88.0000]])
tensor([[575., 409., 648., 492.]])
tensor([[604.0000, 449.3333,  98.6667,  88.0000]])
tensor([[ 93., 775., 181., 840.]])
tensor([[604.0000, 449.3333,  98.6667,  88.0000]])
tensor([[518., 784., 675., 932.]])
tensor([[604.0000, 449.3333,  98.6667,  88.0000]])
tensor([[695., 562., 837., 690.]])
tensor([[604.0000, 449.3333,  98.6667,  88.0000]])
tensor([[677.,  73., 811., 189.]])
tensor([[585.3333, 857.3333, 184.0000, 157.3333]])
tensor([[575., 409., 648., 492.]])
tensor([[585.3333, 857.3333, 184.0000, 157.3333]])
tensor([[ 93., 775., 181., 840.]])
tensor([[585.3333, 857.3333, 184.0000, 157.3333]])
tensor([[518., 784., 675., 932.]])
tensor([[585.3333, 857.3333, 184.0000, 157.3333]])
tensor([[695., 562., 837., 690.]])
tensor([[585.3333, 857.3333, 184.0000, 157.3333]])
tensor([[677.,  73., 811., 189.]])
tensor([[766.6667, 621.3333, 141.3333, 133.3333]])
tensor([[575., 409., 648., 492.]])
tensor([[766.6667, 621.3333, 141.3333, 133.3333]])
ten

In [55]:
len(img_names_list2)

659

In [117]:
act_label_list1 = [ "CAA" if i==0 else "Cored" for i in act_label_list]

In [118]:
iou_list1 = [i.cpu().numpy()[0][0] for i in iou_list]

In [119]:
import pandas as pd
df = pd.DataFrame({"img_name":img_names_list, "pred_label":pred_label_list, "act_label":act_label_list1,"iou":iou_list1})
gt_count_df = pd.DataFrame({"img_name":img_names_list2, "gt":len_gt_labels,"caa_gt":caa_gt, "core_gt":core_gt})
pred_count_df  = pd.DataFrame({"img_name":img_names_list2, "pred":len_pred_labels, "caa_pred":caa_pred, "core_pred":core_pred})

In [120]:
df

,img_name,pred_label,act_label,iou
0,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Coarse-Grained,Cored,0.659698
1,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Coarse-Grained,Cored,0.985075
2,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Coarse-Grained,Cored,0.696774
3,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Coarse-Grained,Cored,0.818548
4,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Diffuse,Cored,0.895522
...,...,...,...,...
850,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Diffuse,CAA,0.068903
851,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,CAA,CAA,0.135311
852,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,CAA,CAA,0.058834
853,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,CAA,CAA,0.165229


In [184]:
df["pred_label1"] = np.where(df["pred_label"].isin(["Coarse-Grained","Cored","Diffuse"]),"Cored",df["pred_label"])

In [185]:
df1 = df[df["iou"]>0]

In [186]:
df1["img_name1"] =df1["img_name"].apply(lambda l: l.split("/")[-1])

In [187]:
df1["match"] = np.where(df1["act_label"]==df1["pred_label1"], 1, 0)

In [188]:
df1["core_match"] = np.where((df1["act_label"]==df1["pred_label1"]) & (df1["act_label"]=="Cored"), 1, 0)

In [189]:
df1["caa_match"] = np.where((df1["act_label"]==df1["pred_label1"]) & (df1["act_label"]=="CAA"), 1, 0)

In [190]:
df1

,img_name,pred_label,act_label,iou,pred_label1,img_name1,match,core_match,caa_match
0,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Coarse-Grained,Cored,0.659698,Cored,cw18-015_0_10_15.jpg,1,1,0
1,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Coarse-Grained,Cored,0.985075,Cored,cw18-015_0_10_15.jpg,1,1,0
2,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Coarse-Grained,Cored,0.696774,Cored,cw17-018_0_8_27.jpg,1,1,0
3,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Coarse-Grained,Cored,0.818548,Cored,cw17-018_0_8_27.jpg,1,1,0
4,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Diffuse,Cored,0.895522,Cored,NA4963UCDtemporalgyri _4G8_0_6_32.jpg,1,1,0
...,...,...,...,...,...,...,...,...,...
850,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,Diffuse,CAA,0.068903,Cored,cw18-014_0_4_2.jpg,0,0,0
851,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,CAA,CAA,0.135311,CAA,cw17-022_0_19_15.jpg,1,0,1
852,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,CAA,CAA,0.058834,CAA,cw17-022_0_19_15.jpg,1,0,1
853,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,CAA,CAA,0.165229,CAA,cw17-022_0_19_15.jpg,1,0,1


In [191]:
from sklearn.metrics import confusion_matrix

confusion_matrix(df1["act_label"], df1["pred_label1"], labels=["Cored", "CAA"])

array([[740,   5],
       [ 37,  73]])

In [192]:
matched = df1.groupby(["img_name"]).agg({"match":"sum","core_match":"sum","caa_match":"sum"}).reset_index()

In [193]:
matched

,img_name,match,core_match,caa_match
0,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,1,0
1,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,1,0
2,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,2,0
3,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,2,0
4,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,3,3,0
...,...,...,...,...
435,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,1,0
436,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,1,0
437,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,3,3,0
438,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,1,0


In [194]:
maatched_gt_df = pd.merge(gt_count_df, matched, on="img_name", how = "left")

In [195]:
maatched_gt_df = pd.merge(maatched_gt_df, pred_count_df, on="img_name", how = "left")

In [196]:
maatched_gt_df

,img_name,gt,caa_gt,core_gt,match,core_match,caa_match,pred,caa_pred,core_pred
0,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,2.0,2.0,0.0,4,0,4
1,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,2.0,2.0,0.0,8,0,8
2,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,6,3,3,1.0,1.0,0.0,3,0,3
3,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,0,2,NaN,NaN,NaN,0,0,0
4,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,5,5,0,1.0,0.0,1.0,3,1,2
...,...,...,...,...,...,...,...,...,...,...
654,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,2,0,2
655,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,7,7,0,0.0,0.0,0.0,1,0,1
656,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,5,0,5
657,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,0,0,0


In [197]:
maatched_gt_df["img_name1"] =maatched_gt_df["img_name"].apply(lambda l: l.split("/")[-1])

In [198]:
maatched_gt_df

,img_name,gt,caa_gt,core_gt,match,core_match,caa_match,pred,caa_pred,core_pred,img_name1
0,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,2.0,2.0,0.0,4,0,4,cw18-015_0_10_15.jpg
1,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,2.0,2.0,0.0,8,0,8,cw17-018_0_8_27.jpg
2,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,6,3,3,1.0,1.0,0.0,3,0,3,NA4963UCDtemporalgyri _4G8_0_6_32.jpg
3,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,0,2,NaN,NaN,NaN,0,0,0,48283-3_UTSW_temporal_6E10_0_6_32.jpg
4,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,5,5,0,1.0,0.0,1.0,3,1,2,cw17-023_0_25_22.jpg
...,...,...,...,...,...,...,...,...,...,...,...
654,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,2,0,2,cw17-018_0_6_7.jpg
655,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,7,7,0,0.0,0.0,0.0,1,0,1,cw18-014_0_4_2.jpg
656,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,5,0,5,NA4998UCDtemporalgyri _4G8_0_8_23.jpg
657,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,0,0,0,NA4916UCDtemporalgyri _4G8_0_18_17.jpg


In [199]:
def map_source(l):
    if l.find("UTSW")!=-1:
        return "UTSW"
    elif l.find("UCD")!=-1:
        return "UCD"
    elif l.find("cw")!=-1:
        return "cw"
    else:
        return "Others"
    

In [200]:
maatched_gt_df["source"] =maatched_gt_df["img_name1"].apply(lambda l: map_source(l))

In [201]:
maatched_gt_df["percentage_match"] = maatched_gt_df["match"]/maatched_gt_df["gt"]

In [202]:
maatched_gt_df["img_name1"].values

array(['cw18-015_0_10_15.jpg', 'cw17-018_0_8_27.jpg',
       'NA4963UCDtemporalgyri _4G8_0_6_32.jpg',
       '48283-3_UTSW_temporal_6E10_0_6_32.jpg', 'cw17-023_0_25_22.jpg',
       'cw18-015_0_20_27.jpg', 'NA4092UCDtemporalgyri _4G8_0_25_19.jpg',
       '48283-3_UTSW_temporal_6E10_0_24_15.jpg',
       '48283-3_UTSW_temporal_6E10_0_7_35.jpg', 'cw15-019_0_0_10.jpg',
       '48283-3_UTSW_temporal_6E10_0_5_12.jpg',
       'NA5012UCDtemporalgyri _4G8_0_8_31.jpg',
       'NA5012UCDtemporalgyri _4G8_0_21_31.jpg', 'cw18-015_0_23_27.jpg',
       'NA4003UCDtemporalgyri _4G8_0_19_10.jpg', 'cw17-023_0_24_33.jpg',
       'cw18-002_0_23_23.jpg', 'cw17-022_0_12_32.jpg',
       'cw18-002_0_10_32.jpg', '51266-3_UTSW_temporal_6E10_0_9_31.jpg',
       'NA4996UCDtemporalgyri _4G8_0_10_23.jpg',
       '48283-3_UTSW_temporal_6E10_0_25_30.jpg', 'cw17-023_0_10_20.jpg',
       '51399-3_UTSW_temporal_6E10_0_13_14.jpg',
       '48308-3_UTSW_temporal_6E10_0_1_14.jpg',
       'NA4963UCDtemporalgyri _4G8_0_11_20.jp

In [203]:
maatched_gt_df["img_name1"] =maatched_gt_df["img_name"].apply(lambda l: l.split("/")[-1])

In [204]:
maatched_gt_df[maatched_gt_df["percentage_match"]>=0.4]

,img_name,gt,caa_gt,core_gt,match,core_match,caa_match,pred,caa_pred,core_pred,img_name1,source,percentage_match
0,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,2.0,2.0,0.0,4,0,4,cw18-015_0_10_15.jpg,cw,0.500000
1,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,2.0,2.0,0.0,8,0,8,cw17-018_0_8_27.jpg,cw,0.500000
8,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,3,0,3,3.0,3.0,0.0,5,0,5,48283-3_UTSW_temporal_6E10_0_7_35.jpg,UTSW,1.000000
10,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,1.0,1.0,0.0,3,0,3,48283-3_UTSW_temporal_6E10_0_5_12.jpg,UTSW,1.000000
13,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,3.0,3.0,0.0,9,0,9,cw18-015_0_23_27.jpg,cw,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,1.0,1.0,0.0,1,0,1,48283-3_UTSW_temporal_6E10_0_23_28.jpg,UTSW,1.000000
651,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,0,2,2.0,2.0,0.0,6,0,6,48308-3_UTSW_temporal_6E10_0_12_18.jpg,UTSW,1.000000
652,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,0,2,1.0,1.0,0.0,2,0,2,51399-3_UTSW_temporal_6E10_0_19_17.jpg,UTSW,0.500000
653,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,1.0,1.0,0.0,4,0,4,50141-3_UTSW_temporal_6E10_0_11_14.jpg,UTSW,1.000000


In [205]:
maatched_gt_df["img_name"].iloc[0]

'/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/images/cw18-015_0_10_15.jpg'

In [206]:
maatched_gt_df[maatched_gt_df["percentage_match"]<0.3]

,img_name,gt,caa_gt,core_gt,match,core_match,caa_match,pred,caa_pred,core_pred,img_name1,source,percentage_match
2,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,6,3,3,1.0,1.0,0.0,3,0,3,NA4963UCDtemporalgyri _4G8_0_6_32.jpg,UCD,0.166667
4,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,5,5,0,1.0,0.0,1.0,3,1,2,cw17-023_0_25_22.jpg,cw,0.200000
44,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,1.0,1.0,0.0,5,0,5,NA4963UCDtemporalgyri _4G8_0_13_7.jpg,UCD,0.250000
83,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,5,0,5,1.0,1.0,0.0,2,0,2,51399-3_UTSW_temporal_6E10_0_20_17.jpg,UTSW,0.200000
94,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,1.0,1.0,0.0,4,0,4,cw18-015_0_10_11.jpg,cw,0.250000
106,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,2,2,1.0,1.0,0.0,6,0,6,cw17-023_0_21_25.jpg,cw,0.250000
113,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,1,3,1.0,1.0,0.0,4,0,4,cw17-022_0_6_14.jpg,cw,0.250000
118,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,5,3,2,1.0,1.0,0.0,5,0,5,cw17-023_0_16_33.jpg,cw,0.200000
135,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,0,4,1.0,1.0,0.0,2,0,2,48283-3_UTSW_temporal_6E10_0_6_34.jpg,UTSW,0.250000
139,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,4,2,2,1.0,1.0,0.0,6,0,6,cw17-022_0_3_19.jpg,cw,0.250000


In [207]:
maatched_gt_df["percentage_match"].mean()

0.7490629004833549

In [208]:
maatched_gt_df["match"].sum()/maatched_gt_df["pred"].sum()

0.3246805111821086

In [209]:
maatched_gt_df["caa_match"].sum()/maatched_gt_df["caa_pred"].sum()

0.9733333333333334

In [210]:
maatched_gt_df["caa_match"].sum()/maatched_gt_df["caa_gt"].sum()

0.2056338028169014

In [211]:
maatched_gt_df["core_match"].sum()/maatched_gt_df["core_pred"].sum()

0.3046521202140799

In [212]:
maatched_gt_df["core_match"].sum()/maatched_gt_df["core_gt"].sum()

0.5808477237048666

In [183]:
maatched_gt_df["percentage_match"].mean()

0.28978484898939444

In [100]:
maatched_gt_df["percentage_match"].max()

2.0

In [489]:
maatched_gt_df["percentage_match"].min()

0.0

In [490]:
maatched_gt_df["match"].sum()/maatched_gt_df["pred"].sum()

0.34347826086956523

In [491]:
maatched_gt_df["caa_match"].sum()/maatched_gt_df["caa_pred"].sum()

0.6842105263157895

In [492]:
maatched_gt_df["caa_match"].sum()/maatched_gt_df["caa_gt"].sum()

0.10985915492957747

In [493]:
maatched_gt_df["core_match"].sum()/maatched_gt_df["core_gt"].sum()

0.4654631083202512

In [494]:
maatched_gt_df["core_match"].sum()/maatched_gt_df["core_pred"].sum()

0.33258553000560853

## Check across sources

In [214]:
cw_data = maatched_gt_df[maatched_gt_df["source"]=="cw"]

In [215]:
cw_data["match"].sum()/cw_data["pred"].sum()

0.36079545454545453

In [216]:
cw_data["match"].sum()/cw_data["gt"].sum()

0.5306406685236769

In [217]:
tmp = cw_data
print(tmp["caa_match"].sum()/tmp["caa_pred"].sum())
print(tmp["caa_match"].sum()/tmp["caa_gt"].sum())
print(tmp["core_match"].sum()/tmp["core_gt"].sum())
print(tmp["core_match"].sum()/tmp["core_pred"].sum())

0.9838709677419355
0.30198019801980197
0.6201550387596899
0.32193158953722334


In [218]:
ucd_data = maatched_gt_df[maatched_gt_df["source"]=="UCD"]
tmp = ucd_data
print(tmp["caa_match"].sum()/tmp["caa_pred"].sum())
print(tmp["caa_match"].sum()/tmp["caa_gt"].sum())
print(tmp["core_match"].sum()/tmp["core_gt"].sum())
print(tmp["core_match"].sum()/tmp["core_pred"].sum())

0.9166666666666666
0.12790697674418605
0.5899280575539568
0.3048327137546468


In [219]:
ucd_data["match"].sum()/ucd_data["gt"].sum()

0.510934393638171

In [220]:
ucd_data["match"].sum()/ucd_data["pred"].sum()

0.3137973137973138

In [221]:
ucd_data

,img_name,gt,caa_gt,core_gt,match,core_match,caa_match,pred,caa_pred,core_pred,img_name1,source,percentage_match
2,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,6,3,3,1.0,1.0,0.0,3,0,3,NA4963UCDtemporalgyri _4G8_0_6_32.jpg,UCD,0.166667
6,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,0,0,0,NA4092UCDtemporalgyri _4G8_0_25_19.jpg,UCD,NaN
11,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,0,2,NaN,NaN,NaN,1,0,1,NA5012UCDtemporalgyri _4G8_0_8_31.jpg,UCD,NaN
12,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,1,0,1,NA5012UCDtemporalgyri _4G8_0_21_31.jpg,UCD,NaN
14,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,6,0,6,NA4003UCDtemporalgyri _4G8_0_19_10.jpg,UCD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,5,0,5,2.0,2.0,0.0,5,0,5,NA4998UCDtemporalgyri _4G8_0_4_16.jpg,UCD,0.400000
645,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,1.0,1.0,0.0,1,0,1,NA4092UCDtemporalgyri _4G8_0_22_14.jpg,UCD,1.000000
647,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,11,0,11,NA4996UCDtemporalgyri _4G8_0_11_24.jpg,UCD,NaN
656,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,5,0,5,NA4998UCDtemporalgyri _4G8_0_8_23.jpg,UCD,NaN


In [222]:
UTSW_data = maatched_gt_df[maatched_gt_df["source"]=="UTSW"]
tmp = UTSW_data
print(tmp["caa_match"].sum()/tmp["caa_pred"].sum())
print(tmp["caa_match"].sum()/tmp["caa_gt"].sum())
print(tmp["core_match"].sum()/tmp["core_gt"].sum())
print(tmp["core_match"].sum()/tmp["core_pred"].sum())

1.0
0.014925373134328358
0.5102639296187683
0.2770700636942675


In [65]:
tmp

,img_name,gt,caa_gt,core_gt,match,core_match,caa_match,pred,caa_pred,core_pred,percentage_match,img_name1,source
3,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,0,2,NaN,NaN,NaN,0,0,0,NaN,48283-3_UTSW_temporal_6E10_0_6_32.jpg,UTSW
7,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,2,0,NaN,NaN,NaN,3,0,3,NaN,48283-3_UTSW_temporal_6E10_0_24_15.jpg,UTSW
8,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,3,0,3,3.0,3.0,0.0,3,0,3,1.0,48283-3_UTSW_temporal_6E10_0_7_35.jpg,UTSW
10,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,0,0,0,NaN,48283-3_UTSW_temporal_6E10_0_5_12.jpg,UTSW
19,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,NaN,NaN,NaN,0,0,0,NaN,51266-3_UTSW_temporal_6E10_0_9_31.jpg,UTSW
...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,6,3,3,3.0,3.0,0.0,7,0,7,0.5,48283-3_UTSW_temporal_6E10_0_16_33.jpg,UTSW
650,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,1,0,1,1.0,1.0,0.0,1,0,1,1.0,48283-3_UTSW_temporal_6E10_0_23_28.jpg,UTSW
651,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,0,2,NaN,NaN,NaN,2,0,2,NaN,48308-3_UTSW_temporal_6E10_0_12_18.jpg,UTSW
652,/home/mahirwar/Desktop/Monika/npsad_data/vivek...,2,0,2,NaN,NaN,NaN,0,0,0,NaN,51399-3_UTSW_temporal_6E10_0_19_17.jpg,UTSW


In [223]:
UTSW_data["match"].sum()/UTSW_data["gt"].sum()

0.42892156862745096

In [224]:
UTSW_data["match"].sum()/UTSW_data["pred"].sum()

0.27821939586645467